In [1]:
from copy import deepcopy
from importlib.metadata import version
import platform

import numpy as np
import mido
from rich import print as rprint
import librosa
from numba import njit

import midii

In [2]:
def test_sample():
    print(midii.sample.dataset)
    print(midii.sample.simple)

In [3]:
def test_midii_simple_print_tracks():
    ma = midii.MidiFile(midii.sample.simple[0])
    ma.print_tracks()

In [4]:
def test_mido_dataset_print_tracks():
    ma = mido.MidiFile(midii.sample.dataset[1])
    ma.print_tracks()

In [5]:
def test_midii_print_tracks():
    try:
        ma = midii.MidiFile(
            midii.sample.dataset[1],
            convert_1_to_0=True,
            lyric_encoding="utf-8",
        )
        ma.quantize(unit="32")
        ma.print_tracks()
    except UnicodeDecodeError:
        ma = midii.MidiFile(
            midii.sample.dataset[1],
            convert_1_to_0=True,
            lyric_encoding="cp949",
        )
        ma.quantize(unit="32")
        ma.print_tracks()

In [6]:
def test_midii_quantize():
    ma = midii.MidiFile(
        midii.sample.dataset[0], convert_1_to_0=True, lyric_encoding="cp949"
    )
    print(np.mean(np.array(ma.times) % 15))
    ma.quantize(unit="32")
    print(np.mean(np.array(ma.times) % 15))

In [7]:
def test_to_json():
    ma = midii.MidiFile(
        midii.sample.dataset[0], convert_1_to_0=True, lyric_encoding="cp949"
    )
    rprint(ma.to_json())
    ma.quantize(unit="32")
    rprint(ma.to_json())

In [8]:
def test_lyrics():
    ma = midii.MidiFile(
        midii.sample.dataset[0], convert_1_to_0=True, lyric_encoding="cp949"
    )
    print(ma.lyrics)

In [9]:
def test_times():
    ma = midii.MidiFile(
        midii.sample.dataset[0], convert_1_to_0=True, lyric_encoding="cp949"
    )
    print(ma.times)
    ma.quantize(unit="32")
    print(ma.times)

In [10]:
def test_EF_w_wo():
    ma = midii.MidiFile(
        midii.sample.dataset[0], convert_1_to_0=True, lyric_encoding="cp949"
    )
    ma2 = deepcopy(ma)
    print(np.cumsum(np.array(ma.times, dtype=np.int64))[-10:])
    ma.quantize(unit="32")
    print(np.cumsum(np.array(ma.times, dtype=np.int64))[-10:])
    ma2.quantize(unit="32", sync_error_mitigation=False)
    print(np.cumsum(np.array(ma2.times, dtype=np.int64))[-10:])

In [11]:
def test_midi_type():
    ma = midii.MidiFile(
        midii.sample.dataset[0], convert_1_to_0=True, lyric_encoding="cp949"
    )
    print(ma.type)

In [12]:
def test_midii_quantization():
    ma = midii.MidiFile(
        midii.sample.dataset[0],
        lyric_encoding="cp949",
        # midii.sample.dataset[0], convert_1_to_0=True, lyric_encoding="cp949"
    )
    ma.quantize(unit="32", sync_error_mitigation=False)
    ma.print_tracks(
        track_limit=None,
        track_list=None,
        print_note_info=False,
    )

In [13]:
def test_midii_quantization_function():
    ticks = [2400, 944, 34, 2, 62]
    unit_tick = midii.beat2tick(midii.NOTE["n/32"].beat, ticks_per_beat=480)
    q, e = midii.quantize(ticks, unit=unit_tick)
    print(q, e)

In [14]:
def test_version():
    pkgs = [
        "mido",
        "rich",
        "ipykernel",
        "matplotlib",
        "pytest",
        "numpy",
        "numba",
    ]
    for pkg in pkgs:
        print(f"{pkg} version:", version(pkg))
    print("Python version:", platform.python_version())

    # mido version: 1.3.3
    # rich version: 14.0.0
    # ipykernel version: 6.29.5
    # matplotlib version: 3.10.1
    # pytest version: 8.3.5
    # numpy version: 2.2.5
    # numba version: 0.61.2
    # Python version: 3.13.1

    # print("mido version:", version("mido"))
    # print("numpy version:", version("numpy"))
    # print("rich version:", version("rich"))
    # print("numba version:", version("numba"))

In [15]:
def test_midii_print_times():
    ma = midii.MidiFile(
        midii.sample.dataset[0], convert_1_to_0=True, lyric_encoding="cp949"
    )
    # ma.print_tracks()
    print(ma.times)
    ma.quantize(unit="64")
    print(ma.times)

In [16]:
def test_standalone_quantize():
    ma = midii.MidiFile(
        midii.sample.dataset[0], convert_1_to_0=True, lyric_encoding="cp949"
    )
    # subset = slice(0, 70)
    subset_last = slice(-33, None)
    unit = midii.beat2tick(
        midii.NOTE["n/32"].beat, ticks_per_beat=ma.ticks_per_beat
    )
    times_q32, error_q32 = midii.quantize(ma.times, unit=unit)
    # times_q64, error_q64 = midii.quantize(
    #     ma.times, unit="64", ticks_per_beat=ma.ticks_per_beat
    # )
    # times_q128, error_q128 = midii.quantize(
    #     ma.times, unit="128", ticks_per_beat=ma.ticks_per_beat
    # )
    # print(ma.times[subset])
    print(ma.times[subset_last])
    ma.quantize(unit="32")
    # print(ma.times[subset])
    # print(times_q32[subset])
    print(ma.times[subset_last], type(ma.times[subset_last]))
    print(times_q32[subset_last], type(times_q32[subset_last]))
    # print(times_q64[subset], error_q64)
    # print(times_q128[subset], error_q128)
    # print(times_q64[subset_last])
    # print(times_q128[subset_last])

In [17]:
def test_divmod(t, u):
    if False:
        # $ hyperfine --warmup 10 -r 200 "python test.py"
        # Benchmark 1: python test.py
        # Time (mean ± σ): 228.3 ms ± 8.0 ms [User: 103.5 ms, System: 90.9 ms]
        # Range (min … max): 215.8 ms … 262.7 ms 200 runs
        q, r = divmod(t, u)
    else:
        # hyperfine --warmup 10 -r 200 "python test.py"
        # Benchmark 1: python test.py
        # Time (mean ± σ): 229.2 ms ± 8.5 ms [User: 104.3 ms, System: 89.7 ms]
        # Range (min … max): 215.4 ms … 275.7 ms    200 runs
        q = t // u
        r = t - q * u  # r = t % u
    return q, r

In [18]:
def test_remainder():
    # for i in range(100_000):
    # r = i % 7
    for i in range(100_000):
        q = i // 7
        r = i - q * 7
    return r

In [19]:
@njit(cache=True, fastmath=True)
def test_remainder_numba():
    for i in range(100_000):
        r = i % 7
    # for i in range(100_000):
    #     q = i // 7
    #     r = i - q * 7
    return r

In [20]:
def test_times_to_frames():
    print(librosa.time_to_frames(0.03125, hop_length=256))
    print(midii.second2frame([0.03125], hop_length=256))
    print(midii.second2frame(0.03125, hop_length=256))

In [21]:
def test_continuous_quantization():
    sampling_rate = 22050
    hop_length = 256
    tick_per_beat = 480
    # time_to_pos = 16
    frames = np.load("ba_05004_+4_a_s01_f_02.npy")
    print("frames", frames[:10], frames.sum())
    seconds = midii.frame2second(
        frames, sr=sampling_rate, hop_length=hop_length
    )
    print("seconds", seconds[-10:], seconds.sum())
    unit_beats = midii.NOTE["n/64"].beat
    unit_ticks = midii.beat2tick(unit_beats, ticks_per_beat=tick_per_beat)
    unit_seconds = mido.tick2second(
        unit_ticks, ticks_per_beat=tick_per_beat, tempo=midii.DEFAULT_TEMPO
    )
    unit_frames = midii.second2frame(
        unit_seconds, sr=sampling_rate, hop_length=hop_length
    )
    print("unit_beats", unit_beats)
    print("unit_ticks", unit_ticks)
    print("unit_seconds", unit_seconds)
    print("unit_frames", unit_frames)
    quantized_seconds, err = midii.quantize(seconds, unit=unit_seconds)
    print(quantized_seconds[-10:], sum(quantized_seconds))
    q_frames = midii.second2frame(
        quantized_seconds, sr=sampling_rate, hop_length=hop_length
    )
    print(q_frames[:10], q_frames.sum())
    q_frames_rosa = librosa.time_to_frames(
        quantized_seconds, sr=sampling_rate, hop_length=hop_length
    )
    print(q_frames_rosa[:10], q_frames_rosa.sum())

In [22]:
def test_seconds_to_frames_loss_comparison():
    sr = midii.DEFAULT_SAMPLING_RATE
    hop_length = midii.DEFAULT_HOP_LENGTH
    frames_per_sec = sr / hop_length

    np.random.seed(42)
    num_durations = 20
    base_frames = np.random.randint(5, 150, size=num_durations)
    fractional_parts = np.random.uniform(0.1, 0.9, size=num_durations)
    ideal_float_frames = base_frames + fractional_parts
    times_in_seconds = ideal_float_frames / frames_per_sec

    print(f"Sampling Rate (sr): {sr} Hz")
    print(f"Hop Length (hop_length): {hop_length} samples")
    print(f"Frames per Second: {frames_per_sec:.4f}")
    # print("\n생성된 Base 정수 프레임:")
    # print(base_frames)
    # print("\n추가된 임의 소수 부분:")
    # print(np.round(fractional_parts, 4))
    print("\nideal frames : Base + Fraction")
    print(np.round(ideal_float_frames, 4))
    print("\nseconds:")
    print(np.round(times_in_seconds, 6))
    # print(f"({num_durations})")
    print(" " * 60)

    target_total_frames_float = np.sum(ideal_float_frames)
    target_total_frames_floor = np.floor(target_total_frames_float).astype(int)
    target_total_frames_round = np.round(target_total_frames_float).astype(int)
    total_fractional_parts = np.sum(fractional_parts)

    print(f"sum of ideal frames: {target_total_frames_float:.4f}")
    print(f"  -> int conversion (floor): {target_total_frames_floor}")
    print(f"  -> int conversion (round): {target_total_frames_round}")
    print(f"(sum of fractional parts: {total_fractional_parts:.4f})")
    print(" " * 60)

    frames_librosa = librosa.time_to_frames(
        times_in_seconds, sr=sr, hop_length=hop_length
    )
    total_frames_librosa = np.sum(frames_librosa)
    diff_librosa_vs_target_floor = (
        total_frames_librosa - target_total_frames_floor
    )
    diff_librosa_vs_target_round = (
        total_frames_librosa - target_total_frames_round
    )

    print("--- librosa.time_to_frames  ---")
    print("converted frames:")
    print(frames_librosa)
    print(f"\ntotal frames: {total_frames_librosa}")
    print(f"(vs ideal floor): {diff_librosa_vs_target_floor} frames")
    print(f"(vs ideal round): {diff_librosa_vs_target_round} frames")
    # print(f"(estimated loss: {total_fractional_parts:.2f} frames)")

    print(" " * 60)

    # frames_optimized = second2frame_optimized(
    frames_optimized = midii.second2frame(
        times_in_seconds, sr=sr, hop_length=hop_length
    )
    total_frames_optimized = np.sum(frames_optimized)
    diff_optimized_vs_target_floor = (
        total_frames_optimized - target_total_frames_floor
    )
    diff_optimized_vs_target_round = (
        total_frames_optimized - target_total_frames_round
    )

    print("--- midii.second2frame ---")
    print("converted frames:")
    print(frames_optimized)
    print(f"\ntotal frames: {total_frames_optimized}")
    print(f"(vs ideal floor): {diff_optimized_vs_target_floor} frames")
    print(f"(vs ideal round): {diff_optimized_vs_target_round} frames")


In [23]:
test_sample()

[WindowsPath('C:/Users/chans/repo/midii/src/midii/sample/dataset/ba_05688_-4_a_s02_m_02.mid'), WindowsPath('C:/Users/chans/repo/midii/src/midii/sample/dataset/ba_09303_+0_a_s02_m_02.mid'), WindowsPath('C:/Users/chans/repo/midii/src/midii/sample/dataset/SINGER_16_10TO29_CLEAR_FEMALE_BALLAD_C0632.mid'), WindowsPath('C:/Users/chans/repo/midii/src/midii/sample/dataset/SINGER_66_30TO49_HUSKY_MALE_DANCE_C2835.mid')]
[WindowsPath('C:/Users/chans/repo/midii/src/midii/sample/simple/test_sample1.mid'), WindowsPath('C:/Users/chans/repo/midii/src/midii/sample/simple/test_sample2.mid'), WindowsPath('C:/Users/chans/repo/midii/src/midii/sample/simple/test_sample3.mid'), WindowsPath('C:/Users/chans/repo/midii/src/midii/sample/simple/test_sample4.mid'), WindowsPath('C:/Users/chans/repo/midii/src/midii/sample/simple/test_sample5.mid'), WindowsPath('C:/Users/chans/repo/midii/src/midii/sample/simple/test_sample6.mid'), WindowsPath('C:/Users/chans/repo/midii/src/midii/sample/simple/test_sample7.mid'), Wind

In [24]:
test_midii_simple_print_tracks()

╭─────────────────────────────────────────────── test_sample1.mid ────────────────────────────────────────────────╮
│ mid file type: 1                                                                                                │
│ ticks per beat: 2                                                                                               │
│ total duration: 16.0                                                                                            │
╰─────────────────────────────── C:\Users\chans\repo\midii\src\midii\sample\simple ───────────────────────────────╯

─────────────────────────────────────────────────── Track 0: "" ───────────────────────────────────────────────────

   0 [set_tempo] BPM=60(1000000)
   1 ┌ E4(64)┐ 2.00/  2.00 time=4  
   2 └ E4(64)┘ 4.00/  6.00 time=8  
   3 ┌ D5(74)┐ 2.00/  8.00 time=4  
   4 └ D5(74)┘ 4.00/ 12.00 time=8  
   5 [end_of_track] 4.00/ 16.00 time=8  
Total secs/time: 16.0/32

In [25]:
test_mido_dataset_print_tracks()

=== Track 0
MetaMessage('track_name', name='Information', time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('key_signature', key='C', time=0)
MetaMessage('set_tempo', tempo=300000, time=0)
MetaMessage('set_tempo', tempo=895522, time=1920)
MetaMessage('time_signature', numerator=2, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=11520)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=960)
MetaMessage('time_signature', numerator=2, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=119040)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=960)
MetaMessage('end_of_track', time=0)
=== Track 1
MetaMessage('track_name', name='Melody', time=0)
Message('note_on', channel=0, note=60, velocity=70, time=0)
MetaMessage('lyrics', text='J '

In [26]:
test_midii_print_tracks()

╭────────────────────────────────────────── ba_09303_+0_a_s02_m_02.mid ───────────────────────────────────────────╮
│ mid file type: 0                                                                                                │
│ ticks per beat: 480                                                                                             │
│ total duration: 255.52824800000107                                                                              │
╰────────────────────────────── C:\Users\chans\repo\midii\src\midii\sample\dataset ───────────────────────────────╯

───────────────────────────────────────────── Track 0: "Information" ──────────────────────────────────────────────

────────────────────────────────────────── Total item num of BPM(200): 2 ──────────────────────────────────────────

╭────────────────────────────────────────── ba_09303_+0_a_s02_m_02.mid ───────────────────────────────────────────╮
│ mid file type: 0                                                                                                │
│ ticks per beat: 480                                                                                             │
│ total duration: 255.52824800000107                                                                              │
╰────────────────────────────── C:\Users\chans\repo\midii\src\midii\sample\dataset ───────────────────────────────╯

───────────────────────────────────────────── Track 0: "Information" ──────────────────────────────────────────────

────────────────────────────────────────── Total item num of BPM(200): 2 ──────────────────────────────────────────

───────────────────────────────────────── Total item num of BPM(67): 820 ──────────────────────────────────────────

   0 [track_name] Information
   1 [time_signature] 4/4
   2 [key_signature] C
   3 [set_tempo] BPM=200(300000)
   4 [track_name] Melody
   5 ┌ C4(60)┐
   6 │   J   │
   7 [set_tempo] 1.20/  1.20 time=1920 BPM=67(895522)
   8 [time_signature] 21.49/ 22.69 time=11520 2/4
   9 [time_signature] 1.79/ 24.48 time=960 4/4
  10 └ C4(60)┘ 3.58/ 28.07 time=1920
  11 ┌ G4(67)┐ 0.56/ 28.63 time=300
  12 │  곱   │
  13 └ G4(67)┘ 0.45/ 29.07 time=240
  14 ┌ G4(67)┐ 0.11/ 29.19 time=60 
  15 │  고   │
  16 └ G4(67)┘ 0.34/ 29.52 time=180
  17 ┌ G4(67)┐
  18 │  희   │
  19 └ G4(67)┘ 1.34/ 30.86 time=720
  20 ┌ G4(67)┐
  21 │  던   │
  22 └ G4(67)┘ 0.45/ 31.31 time=240
  23 ┌ F4(65)┐
  24 │  그   │
  25 └ F4(65)┘ 0.22/ 31.54 time=120
  26 ┌ F4(65)┐
  27 │  손   │
  28 └ F4(65)┘ 0.34/ 31.87 time=180
  29 ┌ E4(64)┐
  30 │  으   │
  31 └ E4(64)┘ 0.22/ 32.10 time=120
  32 ┌ C4(60)┐
  33 │  로   │
  34 └ C4(60)┘ 1.01/ 33.10 time=540
  35 ┌ C5(72)┐ 1.79/ 34.89 time=960
  36 │  넥   │
  37 └ C5(72)┘ 0.34/ 35.23 time=180
  38 ┌ A4(69)┐
  39 │  타   │
  40 └ A4(69)┘ 1.34/ 36.57 time=720
  41 ┌ A4(69)┐
  42 │  이   │
  43 └ A4(69)┘ 0.22/ 36.80 time=120
  44 ┌ B4(71)┐
  45 │  를   │
  46 └ B4(71)┘ 0.56/ 37.36 time=300
  47 ┌ C5(72)┐
  48 │  매   │
  49 └ C5(72)┘ 0.56/ 37.92 time=300
  50 ┌ C5(72)┐
  51 │  어   │
  52 └ C5(72)┘ 0.22/ 38.14 time=120
  53 ┌ B4(71)┐
  54 │  주   │
  55 └ B4(71)┘ 0.34/ 38.48 time=180
  56 ┌ A4(69)┐
  57 │  던   │
  58 └ A4(69)┘ 0.34/ 38.81 time=180
  59 ┌ G4(67)┐
  60 │  때   │
  61 └ G4(67)┘ 1.46/ 40.27 time=780
  62 ┌ E4(64)┐ 1.68/ 41.95 time=900
  63 │  어   │
  64 └ E4(64)┘ 0.22/ 42.17 time=120
  65 ┌ G4(67)┐
  66 │  렴   │
  67 └ G4(67)┘ 0.45/ 42.62 time=240
  68 ┌ G4(67)┐
  69 │  풋   │
  70 └ G4(67)┘ 1.01/ 43.63 time=540
  71 ┌ G4(67)┐
  72 │  이   │
  73 └ G4(67)┘ 0.67/ 44.30 time=360
  74 ┌F#4(66)┐ 0.11/ 44.41 time=60 
  75 │  새   │
  76 └F#4(66)┘ 0.45/ 44.86 time=240
  77 ┌ G4(67)┐
  78 │  앵   │
  79 └ G4(67)┘ 0.22/ 45.08 time=120
  80 ┌ F4(65)┐
  81 │  각   │
  82 └ F4(65)┘ 0.67/ 45.75 time=360
  83 ┌ E4(64)┐
  84 │  나   │
  85 └ E4(64)┘ 0.34/ 46.09 time=180
  86 ┌ D4(62)┐
  87 │  오   │
  88 └ D4(62)┘ 1.34/ 47.43 time=720
  89 ┌C#4(61)┐ 2.46/ 49.89 time=1320
  90 │  여   │
  91 └C#4(61)┘ 0.34/ 50.23 time=180
  92 ┌ D4(62)┐
  93 │  보   │
  94 └ D4(62)┘ 0.22/ 50.45 time=120
  95 ┌ D4(62)┐ 0.11/ 50.57 time=60 
  96 │  그   │
  97 └ D4(62)┘ 0.34/ 50.90 time=180
  98 ┌ D4(62)┐
  99 │  때   │
 100 └ D4(62)┘ 0.34/ 51.24 time=180
 101 ┌ E4(64)┐
 102 │  를   │
 103 └ E4(64)┘ 0.45/ 51.69 time=240
 104 ┌ F4(65)┐
 105 │  기   │
 106 └ F4(65)┘ 0.34/ 52.02 time=180
 107 ┌ E4(64)┐

In [27]:
test_midii_quantization()

╭────────────────────────────────────────── ba_05688_-4_a_s02_m_02.mid ───────────────────────────────────────────╮
│ mid file type: 1                                                                                                │
│ ticks per beat: 120                                                                                             │
│ total duration: 261.4451401249997                                                                               │
╰────────────────────────────── C:\Users\chans\repo\midii\src\midii\sample\dataset ───────────────────────────────╯

───────────────────────────────────────────── Track 0: "Information" ──────────────────────────────────────────────

   0 [track_name] Information
   1 [time_signature] 4/4
   2 [key_signature] Eb
   3 [set_tempo] BPM=200(300000)
   4 [set_tempo] 1.20/  1.20 time=480 BPM=62(967742)
   5 [set_tempo] 79.80/ 81.00 time=31920 BPM=62(967742)
   6 [set_tempo] 0.04/ 81.04 time=15  BPM=61(983607)
   7 [set_tempo] 0.04/ 81.07 time=15  BPM=60(1000000)
   8 [set_tempo] 0.04/ 81.11 time=15  BPM=59(1016949)
   9 [set_tempo] 0.04/ 81.15 time=15  BPM=58(1034483)
  10 [set_tempo] 0.04/ 81.19 time=15  BPM=57(1052632)
  11 [set_tempo] 0.04/ 81.22 time=15  BPM=56(1071429)
  12 [set_tempo] 0.04/ 81.26 time=15  BPM=55(1090909)
  13 [set_tempo] 0.04/ 81.30 time=15  BPM=54(1111111)
  14 [set_tempo] 0.04/ 81.34 time=15  BPM=53(1132075)
  15 [set_tempo] 0.04/ 81.37 time=15  BPM=52(1153846)
  16 [set_tempo] 0.04/ 81.41 time=15  BPM=51(1176471)
  17 [set_tempo] 0.04/ 81.45 time=15  BPM=50(1200000)
  18 [set_tempo] 0.04/ 81.49 time=15  BPM=49(1224490)
  19 [set_tempo] 0.04/ 81.52 time=15  BPM=48(1250000)
  20 [set_tempo] BPM=48(1250000)
  21 [set_tempo] 0.04/ 81.56 time=15  BPM=47(1276596)
  22 [set_tempo] BPM=46(1304348)
  23 [set_tempo] BPM=45(1333333)
  24 [set_tempo] 0.04/ 81.60 time=15  BPM=44(1363636)
  25 [set_tempo] BPM=43(1395349)
  26 [set_tempo] BPM=42(1428571)
  27 [set_tempo] BPM=41(1463415)
  28 [set_tempo] 0.04/ 81.64 time=15  BPM=40(1500000)
  29 [set_tempo] BPM=39(1538462)
  30 [set_tempo] BPM=38(1578947)
  31 [set_tempo] 0.04/ 81.67 time=15  BPM=37(1621622)
  32 [set_tempo] BPM=36(1666667)
  33 [set_tempo] BPM=35(1714286)
  34 [set_tempo] BPM=34(1764706)
  35 [set_tempo] 0.04/ 81.71 time=15  BPM=33(1818182)
  36 [set_tempo] BPM=32(1875000)
  37 [set_tempo] BPM=31(1935484)
  38 [set_tempo] BPM=60(1000000)
  39 [end_of_track]
Total secs/time: 81.7125/32685

──────────────────────────────────────────────── Track 1: "Melody" ────────────────────────────────────────────────

   0 [track_name] Melody
   1 ┌ C4(60)┐
   2 │   J   │
   3 └ C4(60)┘ 10.00/ 10.00 time=2400
   4 ┌ G4(67)┐ 3.94/ 13.94 time=945
   5 │  다   │
   6 └ G4(67)┘ 0.12/ 14.06 time=30 
   7 ┌ G4(67)┐
   8 │  정   │
   9 └ G4(67)┘ 0.25/ 14.31 time=60 
  10 ┌ G4(67)┐
  11 │  했   │
  12 └ G4(67)┘ 0.25/ 14.56 time=60 
  13 ┌ G4(67)┐
  14 │  던   │
  15 └ G4(67)┘ 0.25/ 14.81 time=60 
  16 ┌G#4(68)┐
  17 │  사   │
  18 └G#4(68)┘ 0.19/ 15.00 time=45 
  19 ┌G#4(68)┐
  20 │  람   │
  21 └G#4(68)┘ 0.25/ 15.25 time=60 
  22 ┌G#4(68)┐
  23 │  이   │
  24 └G#4(68)┘ 0.25/ 15.50 time=60 
  25 ┌ G4(67)┐
  26 │  여   │
  27 └ G4(67)┘ 0.31/ 15.81 time=75 
  28 ┌ G4(67)┐
  29 │  나   │
  30 └ G4(67)┘ 0.12/ 15.94 time=30 
  31 ┌ F4(65)┐
  32 │  를   │
  33 └ F4(65)┘ 0.31/ 16.25 time=75 
  34 ┌ F4(65)┐
  35 │  잊   │
  36 └ F4(65)┘ 0.12/ 16.38 time=30 
  37 ┌D#4(63)┐
  38 │  었   │
  39 └D#4(63)┘ 0.25/ 16.62 time=60 
  40 ┌D#4(63)┐
  41 │  나   │
  42 └D#4(63)┘ 0.31/ 16.94 time=75 
  43 ┌ C5(72)┐ 0.75/ 17.69 time=180
  44 │  벌   │
  45 └ C5(72)┘ 0.19/ 17.88 time=45 
  46 ┌ C5(72)┐
  47 │  써   │
  48 └ C5(72)┘ 0.25/ 18.12 time=60 
  49 ┌ C5(72)┐
  50 │  나   │
  51 └ C5(72)┘ 0.25/ 18.38 time=60 
  52 ┌A#4(70)┐
  53 │  를   │
  54 └A#4(70)┘ 0.38/ 18.75 time=90 
  55 ┌ F4(65)┐ 0.25/ 19.00 time=60 
  56 │  잊   │
  57 └ F4(65)┘ 0.12/ 19.12 time=30 
  58 ┌ F4(65)┐
  59 │  어   │
  60 └ F4(65)┘ 0.12/ 19.25 time=30 
  61 ┌ F4(65)┐
  62 │  버   │
  63 └ F4(65)┘ 0.25/ 19.50 time=60 
  64 ┌D#4(63)┐
  65 │  렸   │
  66 └D#4(63)┘ 0.25/ 19.75 time=60 
  67 ┌ F4(65)┐
  68 │  나   │
  69 └ F4(65)┘ 0.62/ 20.38 time=150
  70 ┌ G4(67)┐ 1.31/ 21.69 time=315
  71 │  그   │
  72 └ G4(67)┘ 0.19/ 21.88 time=45 
  73 ┌ G4(67)┐
  74 │  리   │
  75 └ G4(67)┘ 0.25/ 22.12 time=60 
  76 ┌ G4(67)┐
  77 │  움   │
  78 └ G4(67)┘ 0.25/ 22.38 time=60 
  79 ┌ G4(67)┐
  80 │  만   │
  81 └ G4(67)┘ 0.31/ 22.69 time=75 
  82 ┌G#4(68)┐
  83 │  남   │
  84 └G#4(68)┘ 0.12/ 22.81 time=30 
  85 ┌G#4(68)┐
  86 │  겨   │
  87 └G#4(68)┘ 0.25/ 23.06 time=60 
  88 ┌G#4(68)┐
  89 │  놓   │
  90 └G#4(68)┘ 0.31/ 23.38 time=75 
  91 ┌ G4(67)┐
  92 │  고   │
  93 └ G4(67)┘ 0.25/ 23.62 time=60 
  94 ┌ G4(67)┐
  95 │  나   │
  96 └ G4(67)┘ 0.12/ 23.75 time=30 
  97 ┌ F4(65)┐
  98 │  를   │
  99 └ F4(65)┘ 0.31/ 24.06 time=75 
 100 ┌ F4(65)┐
 101 │  잊   │
 102 └ F4(65)┘ 0.12/ 24.19 time=30 
 103 ┌D#4(63)┐
 104 │  었   │
 105 └D#4(63)┘ 0.25/ 24.44 time=60 
 106 ┌D#4(63)┐
 107 │

In [28]:
test_midii_quantize()

3.4230355220667383
0.0


In [29]:
test_to_json()

[
    {'start': 0, 'end': 2400, 'duration': 2400, 'pitch': 60, 'lyric': 'J'},
    {'start': 3344, 'end': 3378, 'duration': 34, 'pitch': 67, 'lyric': '다'},
    {'start': 3380, 'end': 3442, 'duration': 62, 'pitch': 67, 'lyric': '정'},
    {'start': 3445, 'end': 3500, 'duration': 55, 'pitch': 67, 'lyric': '했'},
    {'start': 3503, 'end': 3570, 'duration': 67, 'pitch': 67, 'lyric': '던'},
    {'start': 3574, 'end': 3625, 'duration': 51, 'pitch': 68, 'lyric': '사'},
    {'start': 3628, 'end': 3687, 'duration': 59, 'pitch': 68, 'lyric': '람'},
    {'start': 3689, 'end': 3749, 'duration': 60, 'pitch': 68, 'lyric': '이'},
    {'start': 3751, 'end': 3828, 'duration': 77, 'pitch': 67, 'lyric': '여'},
    {'start': 3830, 'end': 3867, 'duration': 37, 'pitch': 67, 'lyric': '나'},
    {'start': 3869, 'end': 3945, 'duration': 76, 'pitch': 65, 'lyric': '를'},
    {'start': 3948, 'end': 3982, 'duration': 34, 'pitch': 65, 'lyric': '잊'},
    {'start': 3984, 'end': 4044, 'duration': 60, 'pitch': 63, 'lyric': '었'},
    {'start': 4046, 'end': 4125, 'duration': 79, 'pitch': 63, 'lyric': '나'},
    {'start': 4303, 'end': 4350, 'duration': 47, 'pitch': 72, 'lyric': '벌'},
    {'start': 4352, 'end': 4405, 'duration': 53, 'pitch': 72, 'lyric': '써'},
    {'start': 4407, 'end': 4460, 'duration': 53, 'pitch': 72, 'lyric': '나'},
    {'start': 4462, 'end': 4557, 'duration': 95, 'pitch': 70, 'lyric': '를'},
    {'start': 4615, 'end': 4638, 'duration': 23, 'pitch': 65, 'lyric': '잊'},
    {'start': 4641, 'end': 4676, 'duration': 35, 'pitch': 65, 'lyric': '어'},
    {'start': 4678, 'end': 4734, 'duration': 56, 'pitch': 65, 'lyric': '버'},
    {'start': 4737, 'end': 4795, 'duration': 58, 'pitch': 63, 'lyric': '렸'},
    {'start': 4797, 'end': 4948, 'duration': 151, 'pitch': 65, 'lyric': '나'},
    {'start': 5263, 'end': 5304, 'duration': 41, 'pitch': 67, 'lyric': '그'},
    {'start': 5306, 'end': 5368, 'duration': 62, 'pitch': 67, 'lyric': '리'},
    {'start': 5371, 'end': 5426, 'duration': 55, 'pitch': 67, 'lyric': '움'},
    {'start': 5429, 'end': 5511, 'duration': 82, 'pitch': 67, 'lyric': '만'},
    {'start': 5514, 'end': 5548, 'duration': 34, 'pitch': 68, 'lyric': '남'},
    {'start': 5551, 'end': 5612, 'duration': 61, 'pitch': 68, 'lyric': '겨'},
    {'start': 5614, 'end': 5690, 'duration': 76, 'pitch': 68, 'lyric': '놓'},
    {'start': 5692, 'end': 5754, 'duration': 62, 'pitch': 67, 'lyric': '고'},
    {'start': 5756, 'end': 5785, 'duration': 29, 'pitch': 67, 'lyric': '나'},
    {'start': 5788, 'end': 5868, 'duration': 80, 'pitch': 65, 'lyric': '를'},
    {'start': 5871, 'end': 5897, 'duration': 26, 'pitch': 65, 'lyric': '잊'},
    {'start': 5900, 'end': 5966, 'duration': 66, 'pitch': 63, 'lyric': '었'},
    {'start': 5969, 'end': 6067, 'duration': 98, 'pitch': 63, 'lyric': '나'},
    {'start': 6227, 'end': 6277, 'duration': 50, 'pitch': 72, 'lyric': '벌'},
    {'start': 6279, 'end': 6329, 'duration': 50, 'pitch': 72, 'lyric': '써'},
    {'start': 6332, 'end': 6379, 'duration': 47, 'pitch': 72, 'lyric': '나'},
    {'start': 6382, 'end': 6471, 'duration': 89, 'pitch': 70, 'lyric': '를'},
    {'start': 6474, 'end': 6499, 'duration': 25, 'pitch': 70, 'lyric': '잊'},
    {'start': 6501, 'end': 6558, 'duration': 57, 'pitch': 67, 'lyric': '어'},
    {'start': 6561, 'end': 6619, 'duration': 58, 'pitch': 65, 'lyric': '버'},
    {'start': 6622, 'end': 6688, 'duration': 66, 'pitch': 63, 'lyric': '렸'},
    {'start': 6691, 'end': 6821, 'duration': 130, 'pitch': 63, 'lyric': '나'},
    {'start': 7188, 'end': 7223, 'duration': 35, 'pitch': 67, 'lyric': '그'},
    {'start': 7226, 'end': 7271, 'duration': 45, 'pitch': 67, 'lyric': '대'},
    {'start': 7273, 'end': 7341, 'duration': 68, 'pitch': 67, 'lyric': '지'},
    {'start': 7344, 'end': 7427, 'duration': 83, 'pitch': 67, 'lyric': '금'},
    {'start': 7429, 'end': 7472, 'duration': 43, 'pitch': 68, 'lyric': '그'},
    {'start': 7474, 'end': 7523, 'duration': 49, 'pitch': 68, 'lyric': '누'},
    {'start': 7526, 'end': 7585, 'duration': 59, 'pitch': 68, 'lyric':

[
    {'start': 0, 'end': 2400, 'duration': 2400, 'pitch': 60, 'lyric': 'J'},
    {'start': 3345, 'end': 3375, 'duration': 30, 'pitch': 67, 'lyric': '다'},
    {'start': 3375, 'end': 3435, 'duration': 60, 'pitch': 67, 'lyric': '정'},
    {'start': 3450, 'end': 3495, 'duration': 45, 'pitch': 67, 'lyric': '했'},
    {'start': 3510, 'end': 3570, 'duration': 60, 'pitch': 67, 'lyric': '던'},
    {'start': 3570, 'end': 3630, 'duration': 60, 'pitch': 68, 'lyric': '사'},
    {'start': 3630, 'end': 3690, 'duration': 60, 'pitch': 68, 'lyric': '람'},
    {'start': 3690, 'end': 3750, 'duration': 60, 'pitch': 68, 'lyric': '이'},
    {'start': 3750, 'end': 3825, 'duration': 75, 'pitch': 67, 'lyric': '여'},
    {'start': 3825, 'end': 3870, 'duration': 45, 'pitch': 67, 'lyric': '나'},
    {'start': 3870, 'end': 3945, 'duration': 75, 'pitch': 65, 'lyric': '를'},
    {'start': 3945, 'end': 3975, 'duration': 30, 'pitch': 65, 'lyric': '잊'},
    {'start': 3990, 'end': 4050, 'duration': 60, 'pitch': 63, 'lyric': '었'},
    {'start': 4050, 'end': 4125, 'duration': 75, 'pitch': 63, 'lyric': '나'},
    {'start': 4305, 'end': 4350, 'duration': 45, 'pitch': 72, 'lyric': '벌'},
    {'start': 4350, 'end': 4410, 'duration': 60, 'pitch': 72, 'lyric': '써'},
    {'start': 4410, 'end': 4455, 'duration': 45, 'pitch': 72, 'lyric': '나'},
    {'start': 4455, 'end': 4560, 'duration': 105, 'pitch': 70, 'lyric': '를'},
    {'start': 4620, 'end': 4635, 'duration': 15, 'pitch': 65, 'lyric': '잊'},
    {'start': 4635, 'end': 4680, 'duration': 45, 'pitch': 65, 'lyric': '어'},
    {'start': 4680, 'end': 4740, 'duration': 60, 'pitch': 65, 'lyric': '버'},
    {'start': 4740, 'end': 4800, 'duration': 60, 'pitch': 63, 'lyric': '렸'},
    {'start': 4800, 'end': 4950, 'duration': 150, 'pitch': 65, 'lyric': '나'},
    {'start': 5265, 'end': 5310, 'duration': 45, 'pitch': 67, 'lyric': '그'},
    {'start': 5310, 'end': 5370, 'duration': 60, 'pitch': 67, 'lyric': '리'},
    {'start': 5370, 'end': 5430, 'duration': 60, 'pitch': 67, 'lyric': '움'},
    {'start': 5430, 'end': 5505, 'duration': 75, 'pitch': 67, 'lyric': '만'},
    {'start': 5520, 'end': 5550, 'duration': 30, 'pitch': 68, 'lyric': '남'},
    {'start': 5550, 'end': 5610, 'duration': 60, 'pitch': 68, 'lyric': '겨'},
    {'start': 5610, 'end': 5685, 'duration': 75, 'pitch': 68, 'lyric': '놓'},
    {'start': 5685, 'end': 5760, 'duration': 75, 'pitch': 67, 'lyric': '고'},
    {'start': 5760, 'end': 5790, 'duration': 30, 'pitch': 67, 'lyric': '나'},
    {'start': 5790, 'end': 5865, 'duration': 75, 'pitch': 65, 'lyric': '를'},
    {'start': 5865, 'end': 5895, 'duration': 30, 'pitch': 65, 'lyric': '잊'},
    {'start': 5895, 'end': 5970, 'duration': 75, 'pitch': 63, 'lyric': '었'},
    {'start': 5970, 'end': 6060, 'duration': 90, 'pitch': 63, 'lyric': '나'},
    {'start': 6225, 'end': 6270, 'duration': 45, 'pitch': 72, 'lyric': '벌'},
    {'start': 6285, 'end': 6330, 'duration': 45, 'pitch': 72, 'lyric': '써'},
    {'start': 6330, 'end': 6375, 'duration': 45, 'pitch': 72, 'lyric': '나'},
    {'start': 6375, 'end': 6465, 'duration': 90, 'pitch': 70, 'lyric': '를'},
    {'start': 6480, 'end': 6495, 'duration': 15, 'pitch': 70, 'lyric': '잊'},
    {'start': 6495, 'end': 6555, 'duration': 60, 'pitch': 67, 'lyric': '어'},
    {'start': 6555, 'end': 6615, 'duration': 60, 'pitch': 65, 'lyric': '버'},
    {'start': 6615, 'end': 6690, 'duration': 75, 'pitch': 63, 'lyric': '렸'},
    {'start': 6690, 'end': 6825, 'duration': 135, 'pitch': 63, 'lyric': '나'},
    {'start': 7185, 'end': 7230, 'duration': 45, 'pitch': 67, 'lyric': '그'},
    {'start': 7230, 'end': 7275, 'duration': 45, 'pitch': 67, 'lyric': '대'},
    {'start': 7275, 'end': 7335, 'duration': 60, 'pitch': 67, 'lyric': '지'},
    {'start': 7350, 'end': 7425, 'duration': 75, 'pitch': 67, 'lyric': '금'},
    {'start': 7425, 'end': 7470, 'duration': 45, 'pitch': 68, 'lyric': '그'},
    {'start': 7470, 'end': 7530, 'duration': 60, 'pitch': 68, 'lyric': '누'},
    {'start': 7530, 'end': 7590, 'duration': 60, 'pitch': 68, 'lyric'

In [30]:
test_lyrics()

J다정했던사람이여나를잊었나벌써나를잊어버렸나그리움만남겨놓고나를잊었나벌써나를잊어버렸나그대지금그누구를사랑하는가굳은약속변해버렸나예전에는우린서로사랑했는데이젠맘이변해버렸나아이별이그리쉬운가세월가버렸다고이젠나를잊고서멀리멀리떠나가는가아아나는몰랐네그대마음변할주우울난정말몰랐었네오나너하나만으을믿고살았네에에에그대만으을믿었네오네가보고파서어나는어쩌나아아그리우움만쌓이네아이별이그리쉬운가세월가버렸다고이젠나를잊고서멀리멀리떠나가는가아아나는몰랐네그대마음변할주우울난정말몰랐었네오오오난너하나만으을믿고살았네에에그대만으을믿었네오네가아보고파서어나는어쩌나아아그리우움만쌓이네H


In [31]:
test_times()

[0, 0, 0, 0, 0, 0, 0, 480, 1920, 944, 0, 34, 2, 0, 62, 3, 0, 55, 3, 0, 67, 4, 0, 51, 3, 0, 59, 2, 0, 60, 2, 0, 77, 2, 0, 37, 2, 0, 76, 3, 0, 34, 2, 0, 60, 2, 0, 79, 178, 0, 47, 2, 0, 53, 2, 0, 53, 2, 0, 95, 58, 0, 23, 3, 0, 35, 2, 0, 56, 3, 0, 58, 2, 0, 151, 315, 0, 41, 2, 0, 62, 3, 0, 55, 3, 0, 82, 3, 0, 34, 3, 0, 61, 2, 0, 76, 2, 0, 62, 2, 0, 29, 3, 0, 80, 3, 0, 26, 3, 0, 66, 3, 0, 98, 160, 0, 50, 2, 0, 50, 3, 0, 47, 3, 0, 89, 3, 0, 25, 2, 0, 57, 3, 0, 58, 3, 0, 66, 3, 0, 130, 367, 0, 35, 3, 0, 45, 2, 0, 68, 3, 0, 83, 2, 0, 43, 2, 0, 49, 3, 0, 59, 3, 0, 42, 34, 0, 38, 3, 0, 82, 3, 0, 40, 2, 0, 53, 2, 0, 97, 228, 0, 34, 2, 0, 50, 2, 0, 43, 2, 0, 55, 48, 0, 37, 2, 0, 25, 2, 0, 27, 3, 0, 60, 3, 0, 204, 296, 0, 32, 2, 0, 63, 2, 0, 60, 3, 0, 73, 2, 0, 39, 2, 0, 76, 3, 0, 68, 2, 0, 44, 2, 0, 44, 3, 0, 46, 40, 0, 90, 2, 0, 32, 2, 0, 82, 149, 0, 33, 2, 0, 56, 3, 0, 62, 3, 0, 76, 3, 0, 40, 2, 0, 46, 2, 0, 60, 2, 0, 66, 3, 0, 107, 386, 0, 249, 3, 0, 54, 2, 0, 33, 2, 0, 93, 43, 0, 40, 3, 0, 67,

In [32]:
test_EF_w_wo()

[32588 32596 32602 32608 32614 32622 32628 32634 32640 32640]
[32595 32595 32595 32610 32610 32625 32625 32640 32640 32640]
[31995 32010 32010 32010 32010 32025 32025 32025 32025 32025]


In [33]:
test_midi_type()

0


In [34]:
test_version()

mido version: 1.3.3
rich version: 14.0.0
ipykernel version: 6.29.5
matplotlib version: 3.10.1
pytest version: 8.3.5
numpy version: 2.2.5
numba version: 0.61.2
Python version: 3.10.16


In [35]:
test_midii_print_times()

[0, 0, 0, 0, 0, 0, 0, 480, 1920, 944, 0, 34, 2, 0, 62, 3, 0, 55, 3, 0, 67, 4, 0, 51, 3, 0, 59, 2, 0, 60, 2, 0, 77, 2, 0, 37, 2, 0, 76, 3, 0, 34, 2, 0, 60, 2, 0, 79, 178, 0, 47, 2, 0, 53, 2, 0, 53, 2, 0, 95, 58, 0, 23, 3, 0, 35, 2, 0, 56, 3, 0, 58, 2, 0, 151, 315, 0, 41, 2, 0, 62, 3, 0, 55, 3, 0, 82, 3, 0, 34, 3, 0, 61, 2, 0, 76, 2, 0, 62, 2, 0, 29, 3, 0, 80, 3, 0, 26, 3, 0, 66, 3, 0, 98, 160, 0, 50, 2, 0, 50, 3, 0, 47, 3, 0, 89, 3, 0, 25, 2, 0, 57, 3, 0, 58, 3, 0, 66, 3, 0, 130, 367, 0, 35, 3, 0, 45, 2, 0, 68, 3, 0, 83, 2, 0, 43, 2, 0, 49, 3, 0, 59, 3, 0, 42, 34, 0, 38, 3, 0, 82, 3, 0, 40, 2, 0, 53, 2, 0, 97, 228, 0, 34, 2, 0, 50, 2, 0, 43, 2, 0, 55, 48, 0, 37, 2, 0, 25, 2, 0, 27, 3, 0, 60, 3, 0, 204, 296, 0, 32, 2, 0, 63, 2, 0, 60, 3, 0, 73, 2, 0, 39, 2, 0, 76, 3, 0, 68, 2, 0, 44, 2, 0, 44, 3, 0, 46, 40, 0, 90, 2, 0, 32, 2, 0, 82, 149, 0, 33, 2, 0, 56, 3, 0, 62, 3, 0, 76, 3, 0, 40, 2, 0, 46, 2, 0, 60, 2, 0, 66, 3, 0, 107, 386, 0, 249, 3, 0, 54, 2, 0, 33, 2, 0, 93, 43, 0, 40, 3, 0, 67,

In [36]:
test_standalone_quantize()

[8, 8, 8, 8, 8, 8, 10, 8, 8, 8, 8, 8, 8, 6, 8, 6, 6, 8, 6, 6, 6, 8, 6, 6, 8, 6, 6, 6, 8, 6, 6, 6, 0]
[0, 15, 0, 15, 0, 15, 15, 0, 15, 0, 15, 0, 15, 0, 15, 0, 0, 15, 0, 15, 0, 15, 0, 15, 0, 0, 15, 0, 15, 0, 15, 0, 0] <class 'list'>
[0, 15, 0, 15, 0, 15, 15, 0, 15, 0, 15, 0, 15, 0, 15, 0, 0, 15, 0, 15, 0, 15, 0, 15, 0, 0, 15, 0, 15, 0, 15, 0, 0] <class 'list'>


In [37]:
test_divmod(100, 18)

(5, 10)

In [38]:
test_remainder()

4

In [39]:
test_remainder_numba()

4

In [40]:
test_midii_quantization_function()

[2400, 960, 0, 0, 60] 22


In [41]:
test_times_to_frames()

2
[3]
3


In [42]:
test_continuous_quantization()

frames [1672  156    7   26   10   62    5   73  143   12] 19750
seconds [0.05804989 0.69659864 0.06965986 0.12770975 0.40634921 0.05804989
 0.55727891 0.1044898  3.49460317 5.97913832] 229.297052154195
unit_beats 0.0625
unit_ticks 30
unit_seconds 0.03125
unit_frames 3
[0.06249999999999999, 0.6875, 0.0625, 0.125, 0.40625, 0.0625, 0.5625, 0.09375, 3.5, 6.0] 229.3125
[1672  156    8   24   11   62    5   73  143   13] 19751
[1671  156    8   24   10   61    5   72  142   13] 19538


In [43]:
test_seconds_to_frames_loss_comparison()

Sampling Rate (sr): 22050 Hz
Hop Length (hop_length): 512 samples
Frames per Second: 43.0664

ideal frames : Base + Fraction
[107.594   97.5893  19.1057 111.1184  76.5198  25.4199 107.1373 126.879
  79.2862  92.1725 121.5947 104.406  108.8866 135.4734  57.788    6.6442
  92.4604  42.1106 134.8538  25.5506]

seconds:
[2.498327 2.26602  0.443632 2.580165 1.776787 0.590249 2.487724 2.946125
 1.841022 2.140241 2.823424 2.424302 2.528342 3.145686 1.341834 0.154279
 2.146926 0.977807 3.131298 0.593284]
                                                            
sum of ideal frames: 1672.5904
  -> int conversion (floor): 1672
  -> int conversion (round): 1673
(sum of fractional parts: 9.5904)
                                                            
--- librosa.time_to_frames  ---
converted frames:
[107  97  19 111  76  25 107 126  79  92 121 104 108 135  57   6  92  42
 134  25]

total frames: 1663
(vs ideal floor): -9 frames
(vs ideal round): -10 frames
                                 